In [43]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

In [44]:
df = spark.read.csv('data-2.csv',header=True,escape="\"")

In [45]:
df.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [46]:
df.filter(df['UnitPrice'] > 7.95).show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536367|    22622|BOX OF VINTAGE AL...|       2| 12/1/2010 8:34|     9.95|     13047|United Kingdom|
|   536370|     POST|             POSTAGE|       3| 12/1/2010 8:45|       18|     12583|        France|
|   536374|    21258|VICTORIAN SEWING ...|      32| 12/1/2010 9:09|    10.95|     15100|United Kingdom|
|  C536379|        D|            Discount|      -1| 12/1/2010 9:41|     27.5|     14527|United Kingdom|
|   536382|    22839|3 TIER CAKE TIN G...|       2| 12/1/2010 9:45|    14.95|     16098|United Kingdom|
|   536382|    22838|3 TIER CAKE TIN R...|       2| 12/1/2010 9:45|    14.95|     16098|United Kingdom|
|   536382|    22783|SET 3 WICKER OVAL...|       4| 12/1/2010 9:

In [47]:
from pyspark.sql.functions import *

df.groupBy('Country').agg(countDistinct('CustomerID').alias('num_customers')).orderBy(desc('num_customers')).show()

+---------------+-------------+
|        Country|num_customers|
+---------------+-------------+
| United Kingdom|         3950|
|        Germany|           95|
|         France|           87|
|          Spain|           31|
|        Belgium|           25|
|    Switzerland|           21|
|       Portugal|           19|
|          Italy|           15|
|        Finland|           12|
|        Austria|           11|
|         Norway|           10|
|        Denmark|            9|
|Channel Islands|            9|
|      Australia|            9|
|    Netherlands|            9|
|         Sweden|            8|
|         Cyprus|            8|
|          Japan|            8|
|         Poland|            6|
|         Greece|            4|
+---------------+-------------+
only showing top 20 rows



In [48]:
df.createOrReplaceTempView('invoices')

In [49]:
spark.sql('SELECT * FROM invoices').show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [50]:
spark.sql(
"""
SELECT
    Country, COUNT(DISTINCT CustomerID) as num_customers
FROM invoices
GROUP BY 1
ORDER BY 2 DESC
"""
).show()

+---------------+-------------+
|        Country|num_customers|
+---------------+-------------+
| United Kingdom|         3950|
|        Germany|           95|
|         France|           87|
|          Spain|           31|
|        Belgium|           25|
|    Switzerland|           21|
|       Portugal|           19|
|          Italy|           15|
|        Finland|           12|
|        Austria|           11|
|         Norway|           10|
|        Denmark|            9|
|Channel Islands|            9|
|      Australia|            9|
|    Netherlands|            9|
|         Sweden|            8|
|         Cyprus|            8|
|          Japan|            8|
|         Poland|            6|
|         Greece|            4|
+---------------+-------------+
only showing top 20 rows



In [51]:
spark.sql(
"""
WITH country_count AS (
    SELECT
        Country, COUNT(DISTINCT CustomerID) as num_customers
    FROM invoices
    GROUP BY 1
    ORDER BY 2 DESC
)

SELECT 
    Country AS country, 
    num_customers,
    ROW_NUMBER() OVER( ORDER BY num_customers DESC) as row_number,
    RANK() OVER(ORDER BY num_customers DESC) as rank,
    DENSE_RANK() OVER(ORDER BY num_customers DESC) as dense_rank
FROM country_count
"""
).show()

+---------------+-------------+----------+----+----------+
|        country|num_customers|row_number|rank|dense_rank|
+---------------+-------------+----------+----+----------+
| United Kingdom|         3950|         1|   1|         1|
|        Germany|           95|         2|   2|         2|
|         France|           87|         3|   3|         3|
|          Spain|           31|         4|   4|         4|
|        Belgium|           25|         5|   5|         5|
|    Switzerland|           21|         6|   6|         6|
|       Portugal|           19|         7|   7|         7|
|          Italy|           15|         8|   8|         8|
|        Finland|           12|         9|   9|         9|
|        Austria|           11|        10|  10|        10|
|         Norway|           10|        11|  11|        11|
|        Denmark|            9|        12|  12|        12|
|Channel Islands|            9|        13|  12|        12|
|      Australia|            9|        14|  12|        1

In [52]:
# doing the same with the functions instead of SQL


from pyspark.sql import Window

window = Window.orderBy(col("num_customers").desc())
country_customers = df.groupBy('Country').agg(countDistinct('CustomerID').alias('num_customers'))

country_customers \
.withColumn("row_number", row_number().over(window)) \
.withColumn("rank", rank().over(window)) \
.withColumn("dense_rank", dense_rank().over(window)) \
.show()

+---------------+-------------+----------+----+----------+
|        Country|num_customers|row_number|rank|dense_rank|
+---------------+-------------+----------+----+----------+
| United Kingdom|         3950|         1|   1|         1|
|        Germany|           95|         2|   2|         2|
|         France|           87|         3|   3|         3|
|          Spain|           31|         4|   4|         4|
|        Belgium|           25|         5|   5|         5|
|    Switzerland|           21|         6|   6|         6|
|       Portugal|           19|         7|   7|         7|
|          Italy|           15|         8|   8|         8|
|        Finland|           12|         9|   9|         9|
|        Austria|           11|        10|  10|        10|
|         Norway|           10|        11|  11|        11|
|        Denmark|            9|        12|  12|        12|
|Channel Islands|            9|        13|  12|        12|
|      Australia|            9|        14|  12|        1